In [1]:
import sys, json
from pathlib import Path
import numpy as np
import plotly.graph_objects as go

sys.path.append("../")
from bias_steering.data.load_dataset import load_dataframe_from_json

%load_ext autoreload
%autoreload 2

In [2]:
COLORS = {
    "MD": '#1F77B4', "WMD": '#FF7F0E', 
    "before": "#4C78A8", "after": "#E45756"
}

def load_json_file(filepath):
    return json.load(open(filepath, "r"))


def plot_projection(
    projections, bias_scores, width=350, height=300, 
    x_range=None, y_range=None, title_text=None
):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=projections, y=bias_scores, mode="markers", marker_color=COLORS["before"], showlegend=False))
    fig.add_shape(type="line", xref="paper", yref="paper", x0=0, y0=0, x1=1, y1=1,
        line=dict(color="#66AA00", width=3, dash="dash"), layer="below"
    )
    
    fig.update_layout(
        title=dict(text=title_text, font=dict(size=18), x=0.55, y=0.97),
        plot_bgcolor='white', width=width, height=height, margin=dict(l=20, r=15, t=25, b=20), 
        font=dict(size=13), legend_title_font=dict(size=14), title_font=dict(size=14)
    )
    fig.update_yaxes(
        title_text="Disparity Score", title_standoff=4,
        title_font=dict(size=15), tickfont=dict(size=13),
        mirror=True, showgrid=True, gridcolor='darkgrey', 
        zeroline = True, zerolinecolor='darkgrey', 
        showline=True, linewidth=1, linecolor='darkgrey',
        range=y_range
    )
    fig.update_xaxes(
        title_text="Projection", title_standoff=5, 
        title_font=dict(size=15), tickfont=dict(size=13),
        mirror=True, showgrid=True, gridcolor='darkgrey', 
        zeroline = True, zerolinecolor='darkgrey', 
        showline=True, linewidth=1, linecolor='darkgrey',
        range=x_range
    )
    return fig


def plot_debias(
    bias_scores, baseline_bias, projections, width=360, height=300, 
    x_range=None, y_range=None, showlegend=True, title_text=None,
    legend_x=0.02, legend_y=0.98, opacity=0.8, 
    legend_font_size=15, title_font_size=17,
):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=projections, y=baseline_bias, mode="markers", 
        marker_color=COLORS["before"], marker_size=5, 
        name="before", showlegend=showlegend
    ))
    fig.add_trace(go.Scatter(
        x=projections, y=bias_scores, mode="markers", 
        marker_color=COLORS["after"], marker_size=5, 
        name="after", showlegend=showlegend, opacity=opacity
    ))
    
    fig.update_layout(
        width=width, height=height,
        margin=dict(l=20, r=15, t=25, b=20),
        font=dict(size=14), plot_bgcolor='white', 
        title=dict(text=title_text, font=dict(size=title_font_size), x=0.55, y=0.98),
        legend=dict(
            yanchor="top", y=legend_y, 
            xanchor="left", x=legend_x,
            bordercolor="darkgrey", borderwidth=1, 
            font=dict(size=legend_font_size)
        ),
    )
    fig.update_xaxes(
        title_text="Projection", title_standoff=5,
        title_font=dict(size=16), tickfont=dict(size=13),
        mirror=True, showgrid=True, gridcolor='darkgrey',
        zeroline = True, zerolinecolor='darkgrey',
        showline=True, linewidth=1, linecolor='darkgrey', 
        range=x_range
    )
    fig.update_yaxes(
        title_text="Disparity Score", title_standoff=4,
        title_font=dict(size=16), tickfont=dict(size=13),
        mirror=True, showgrid=True, gridcolor='darkgrey', 
        zeroline = True, zerolinecolor='darkgrey',
        showline=True, linewidth=1, linecolor='darkgrey', 
        range=y_range
    )
    return fig

## Mistral-Nemo-Instruct

In [10]:
artifact_dir = Path("../runs_gender/Mistral-Nemo-Instruct-2407")
val_data = load_dataframe_from_json(artifact_dir / "datasplits/val.json")
baseline_bias = (val_data["pos_prob"] - val_data["neg_prob"]).to_numpy()
top_layer = load_json_file(artifact_dir / "validation/top_layers.json")[0]["layer"]
projections = np.load(artifact_dir / "validation/projections.npy")[top_layer]

for layer_result in load_json_file(artifact_dir / "validation/debiased_scores.json"):
    if layer_result["layer"] == top_layer:
        debiased_scores = layer_result["bias_scores"]
        break

In [11]:
fig = plot_projection(projections, baseline_bias)
fig.show()
fig = plot_debias(debiased_scores, baseline_bias, projections)
fig.show()

## Qwen-7B-Chat

In [12]:
artifact_dir = Path("../runs_gender/Qwen-7B-Chat")
val_data = load_dataframe_from_json(artifact_dir / "datasplits/val.json")
baseline_bias = (val_data["pos_prob"] - val_data["neg_prob"]).to_numpy()
top_layer = load_json_file(artifact_dir / "validation/top_layers.json")[0]["layer"]
projections = np.load(artifact_dir / "validation/projections.npy")[top_layer]

for layer_result in load_json_file(artifact_dir / "validation/debiased_scores.json"):
    if layer_result["layer"] == top_layer:
        debiased_scores = layer_result["bias_scores"]
        break

In [13]:
fig = plot_projection(projections, baseline_bias)
fig.show()
fig = plot_debias(debiased_scores, baseline_bias, projections)
fig.show()

## Qwen-1.8B-Chat

In [14]:
artifact_dir = Path("../runs_gender/Qwen-1_8B-chat")
val_data = load_dataframe_from_json(artifact_dir / "datasplits/val.json")
baseline_bias = (val_data["pos_prob"] - val_data["neg_prob"]).to_numpy()
top_layer = load_json_file(artifact_dir / "validation/top_layers.json")[0]["layer"]
projections = np.load(artifact_dir / "validation/projections.npy")[top_layer]

for layer_result in load_json_file(artifact_dir / "validation/debiased_scores.json"):
    if layer_result["layer"] == top_layer:
        debiased_scores = layer_result["bias_scores"]
        break

In [15]:
fig = plot_projection(projections, baseline_bias)
fig.show()
fig = plot_debias(debiased_scores, baseline_bias, projections)
fig.show()